# Orange Pi GPIO 串口通讯
## ——使用 Linux 原生工具测试 与 Nodejs SerialPort 开发的过程

**注1**：执行过程中，遇到诸如：

- git 没安装的问题，就自己安装一下，或者

- 权限不足，就设置下权限，又或者

- 文件夹不存在，就自己建一下

- apt 软件源问题，自己设置一下软件源

- npm 源问题或代理问题，自己设置一下源或代理

- 等等

本文不再赘述。

**注2**：请务必边执行变观察输出，及时发现异常并解决。

**前提**：

- 切换到一个专门的文件夹下工作：以 `cd /home/you/serialport` 为例；（注：you 是指用户名，如果不存在该用户，需要 adduser ！）

- 串口以 `/dev/ttyS1` 为例。


## 目录

　　**[1. 安装 gcc 4.8 和 g++ 4.8](### 1. 安装 gcc 4.8 和 g++ 4.8)**

　　**[2. 启用 GPIO 串口](### 2. 启用 GPIO 串口)**

　　**[3. 测试 GPIO 串口](### 3. 测试 GPIO 串口)**

　　**[4. 安装 nodejs](### 4. 安装 nodejs)**

　　**[5. 安装 serialport ](### 5. 安装 serialport )**

　　**[6. 开发示例](### 6. 开发示例)**


### 1. 安装 gcc 4.8 和 g++ 4.8

- 请按照这个步骤：[GCC 4.8 ON RASPBERRY PI WHEEZY](https://somewideopenspace.wordpress.com/2014/02/28/gcc-4-8-on-raspberry-pi-wheezy/)


### 2. 启用 GPIO 串口

- 查看是否已启用：`ls /dev/tty*` 是否有 `/dev/ttyS1` 如果有则已启用，跳到第 2 步。否则：

- 安装编译依赖：`apt-get install -y libusb-1.0-0-dev`；

- 安装工具链：`git clone git://github.com/linux-sunxi/sunxi-tools.git`；

- 编译工具链：`cd sunxi-tools` 然后 `make`；

- 复制：`cp /boot/script.bin ./script.bin`，不要影响源文件即可；

- 备份：`cp ./script.bin ./script.bin.bkup`，保险起见；

- 转换格式：`./sunxi-tools/bin2fex ./script.bin > ./script.fex` （出现 warning 可以忽略）

- 编辑：`nano ./script.fex` 找到如下：

```bash
[uart1]
uart_used = 0
uart_port = 1
uart_type = 4
uart_tx = port:PG06<2><1><default><default>
uart_rx = port:PG07<2><1><default><default>
uart_rts = port:PG08<2><1><default><default>
uart_cts = port:PG09<2><1><default><default>
```
改为：
（注：其中 uart_type 指的是 串口通讯的线制。这里采用的是 2 线制度，所以删掉了后面两行。如果你用的是 4 线制，则保留。）
```bash
[uart1]
uart_used = 1
uart_port = 1
uart_type = 2
uart_tx = port:PG06<2><1><default><default>
uart_rx = port:PG07<2><1><default><default>
```
保存；

- 转回格式：`./sunxi-tools/fex2bin ./script.fex > ./script.bin`（出现 warning 可以忽略）‘

- 覆盖原文：`cp ./script.bin /boot/script/bin`；

- 重启：`reboot`；

- 查看是否已启用：`ls /dev/tty*` 是否有 `/dev/ttyS1` 如果有则已启用，跳到第 2 步。否则：异常。


### 3. 测试 GPIO 串口

- `cat /dev/ttyS1` 以接收来自串口 S1 的数据（注：串口数据一旦被取走，就没有了，意味着一个串口只有一个程序能用来接收，先到先得）

- `echo '要发送的数据' > /dev/ttyS1` 以发送数据到串口 S1

注：可以开两个窗口，便于观察。


### 4. 安装 nodejs

注：版本以 8.11.3 为例。

- 下载：`wget https://nodejs.org/dist/latest-v8.x/node-v8.11.3-linux-armv7l.tar.gz`；

- 解压：`tar xvzf node*`；

- 安装：`cp -a node*/* /opt/node`；

- 设置 root 用户的路径：

```bash
echo "PATH=$PATH:/opt/node/bin" >> /root/.profile
echo "export PATH" >> /root/.profile
PATH=$PATH:/opt/node/bin
export PATH
```

- 设置 you 用户的路径：

```bash
echo "PATH=$PATH:/opt/node/bin" >> /you/.profile
echo "export PATH" >> /you/.profile
PATH=$PATH:/opt/node/bin
export PATH
```

- 检验：`node -v` 无异常 然后 `npm -v` 也无异常；则成功，否则：异常。


### 5. 安装 serialport 

- 安装：`npm install serialport` （注意，最好不要以 root 身份安装）。


### 6. 开发示例

- 官方网站：[node-serialport Github](https://github.com/node-serialport/node-serialport#opening-a-port) 和 [node-serialport Document](https://node-serialport.github.io/node-serialport/)


- 开发示例：（另一方可以用 `cutecom` 进行相应的收发数据）

```javascript
function startSending() {
  var time = 0;

  console.log('started')

  setTimeout(function again() {
    console.log('again');
    port.write(String(time)+'\n', function(err) {
      console.log('sent')
      if (err) {
        return console.log('Error on write: ', err.message);
      }
      console.log('message written' + time);
      time++;
      setTimeout(again, 1000);
    });
  }, 1000);
};

var SerialPort = require('serialport');
var port = new SerialPort('/dev/ttyS1', {
  baudRate: 115200
});

port.on('data', function (data) {
  console.log('Data:', data.toString("hex"));
});

// Read data that is available but keep the stream from entering "flowing mode"
port.on('readable', function () {
  console.log('Data:', port.read());
});

startSending();

```

`*END*`